In [ ]:
%pip install google-genai
%pip install matplotlib
%pip install os
%pip install dotenv

In [ ]:
from google import genai
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
from dotenv import load_dotenv
from google.genai import types
from io import BytesIO
from PIL import Image
from IPython.display import Markdown

In [ ]:
def create_genai_client():
    import os
    
    cloud_api_key = os.getenv("GOOGLE_CLOUD_API_KEY")
    if not cloud_api_key:
        raise ValueError("GOOGLE_CLOUD_API_KEY not found in .env file")
    
    client = genai.Client(
        vertexai=True, 
        api_key=cloud_api_key, 
    )

    return client

In [ ]:
load_dotenv()

# Configure the client with your API key
client = create_genai_client()

tools = [types.Tool(google_search=types.GoogleSearch())]

In [ ]:
def load_image(image_path: str):
    try:
        img = mpimg.imread(image_path)
        plt.imshow(img)
        plt.axis('off')
        plt.show()
    except FileNotFoundError:
        print(f"Error: The file at '{image_path}' was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")
        
def print_token_usage(response: types.GenerateContentResponse):
    if response and response.usage_metadata:
        usage_metadata = response.usage_metadata
        input_token_count = usage_metadata.prompt_token_count
        output_token_count = usage_metadata.candidates_token_count
        total_token_count = usage_metadata.total_token_count
        thought_token_count = usage_metadata.thoughts_token_count
        cached_token_count = usage_metadata.cached_content_token_count
        print(f"Input: {input_token_count}, Output: {output_token_count}, Thought: {thought_token_count}, Cached: {cached_token_count} Total: {total_token_count}")
                
def generate_image(prompt: str, image_config: types.ImageConfig | None = None):
    response = client.models.generate_content(
        model="gemini-3-pro-image-preview",
        contents=[types.Content(
            role="user",
            parts=[types.Part(text=prompt)]
        )],
        config=types.GenerateContentConfig(
            response_modalities=['TEXT', 'IMAGE'],
            thinking_config=types.ThinkingConfig(
                include_thoughts=True,
                thinking_budget=512,
            ),
            tools=tools,
            image_config=image_config
        )
    )

    image_bytes: bytes | None = None
    print_token_usage(response)
    if response.candidates and response.candidates[0].content and response.candidates[0].content.parts:
        for part in response.candidates[0].content.parts:
            if part.thought and part.text:
                display(Markdown(f"Thought Summary:\n {part.text}"))
            elif part.text:
                print("Text: ", part.text)
            elif part.inline_data:
                image_bytes = part.inline_data.data
       
    return image_bytes

def save_and_show_image(image_bytes: bytes, file_name="./image.png"):
    image = Image.open(BytesIO(image_bytes)) if image_bytes else None
    if image:
        plt.imshow(image)
        plt.axis('off')
        plt.show()
        image.save(file_name)

In [ ]:
image_bytes = generate_image(prompt="""
    Please search for the latest confirmed weather forecast for Taiwan for the dates listed below in 2025. specifically looking at Taipei and Taoyuan.
    The dates and locations are:
    1. Taipei: November 26, 27, 28, and 29.
    2. Taoyuan: November 30 and December 1.
    
    November has 30 days, so avoid doing weather forecast for November 31.
    Then generate a weather forecast image, add appropriate clothing on each day and in Traditional Chinese.                      
    
    Constraints: Do not add persons or background to the image                         
""")
save_and_show_image(image_bytes=image_bytes, file_name='./weather_forecast.png')

In [ ]:
image_bytes = generate_image(prompt="""
    Please generate a vertical movie-style travel poster titled "TAIPEI 2025" in Traditional Chinese. 
    The composition is a dynamic vertical montage illustrating a journey. 
    Please use Google Search tool to find out how the landmarks look like and include them in the poster.

    Visual Elements (blended from bottom to top):
    1. Bottom Section (City & Tech): A bustling street scene featuring "Tenlong Computer Books" and stacks of coding books, merging into the bright neon lights of Ximending and the towering Taipei 101 at night.
    2. Middle Section (Nature & Relax): Steam rising from a traditional Beitou Hot Spring bath set against the lush green backdrop of Yangmingshan mountains.  Please display Beitou and Yangmingshan National Park in Traditional Chinese. 
    3. Top Section (Action & Return): A dynamic female pro-wrestling ring scene (Diana Wrestling) with a small flying Japan flag acting as the climax, Azure AI Logo over 國立臺灣大學社會科學院, with a stylized airplane flying overhead towards a silhouette of the Hong Kong skyline in the clouds.
    4. The female wreslers must look Japanese or Taiwanese.
    
    Style: Vibrant semi-realistic digital art, high saturation, distinct color zones for each location, cinematic lighting, highly detailed.
""", 
image_config=types.ImageConfig(
    aspect_ratio="9:16",
    image_size="2K"
))
save_and_show_image(image_bytes=image_bytes, file_name='./poster.png')